# 01 — From C to C++: The Toolchain 🛠️

## 🎯 Goal
As a C programmer, you know how code transforms from text to binary (Preprocessing → Compilation → Assembly → Linking). This does not change in C++. 

However, the tools and the file organization do change to accommodate C++'s complexity (Templates, Name Mangling, Standard Library).

In this notebook, we will cover:
1. g++ vs gcc
2. File Extensions (.hpp vs .h)
3. The Header/Source Split in C++
4. Modern Build Systems (Why CMake?)

---

## 1. The Compiler: g++

In C, you use gcc. In C++, you generally use g++ (or clang++).

### ❓ Why can't I just use gcc for C++ files?
You actually can, but you shouldn't. 

* gcc treats files ending in .cpp as C++ files (correctly).
* HOWEVER, gcc does not automatically link the C++ Standard Library (libstdc++ or libc++).

If you try to compile C++ object code with gcc, you will likely get undefined reference errors for symbols like std::cout or std::string.

### The Command Line
Just like C, you can compile single files. Note the -std=c++17 flag. C++ evolves faster than C; so always specify the version.

- The C Way

```bash
gcc main.c -std=c99 -o app
```
- The C++ Way

```bash
g++ main.cpp -std=c++17 -Wall -Wextra -o app
```

### 🧪 Interactive Check: The "Hello World" Difference

In C, you rely on stdio.h and printf. In C++, we use the Input/Output Stream library (iostream).

Notice:
1. No .h in the standard header include (\<iostream\>, not <iostream.h>).
2. std:: namespace prefix.
3. << operator for output.

In [3]:
// Run this cell (Shift + Enter)
#include <iostream>

int main() {
    int year = 2023;
    // Type-safe printing. No %d formatting codes required.
    std::cout << "Hello from C++! The year is " << year << std::endl;
    return 0;
}

main();

Hello from C++! The year is 2023


---

## 2. File Extensions & Headers (.hpp)

In C, the convention is strict: .c for source, .h for headers.

In C++, it's a bit looser, but here is the Professional Standard:

| Type | Extension | Note |
| :--- | :--- | :--- |
| Source | .cpp | Avoid .C or .cxx if possible for consistency. |
| Header | .hpp | Distinguishes C++ headers (which may contain templates/classes) from C headers. |

### ⚠️ The "extern C" Issue
C++ compilers "mangle" function names to support Overloading (having two functions with the same name but different arguments).

If you try to link a C object file with a C++ object file, the linker won't find the names. This is why you often see this in legacy headers:

```cpp
extern "C" {
    void my_c_function();
}
```

This tells the C++ compiler: "Don't mangle this name, look for it exactly as it is."

---

## 3. The Build System: Entering CMake

Coming from C, you likely love (or hate) Makefiles.

In C++, dependencies are more complex. Linking OpenMP, CUDA, or Boost in a Makefile requires complex flag management. CMake solves this by abstracting the build process.

CMake is not a compiler. It is a Build System Generator. 
It reads a CMakeLists.txt file and generates a Makefile for you.

### Structure of a Modern C++ Project

We typically organize files like this (see the project/ folder in this directory):

```text
project/
├── CMakeLists.txt       <-- The Recipe
├── include/
│   └── simple_math.hpp  <-- The Declaration (.hpp)
├── src/
│   ├── simple_math.cpp  <-- The Implementation (.cpp)
│   └── main.cpp         <-- The Entry Point
└── build/               <-- We compile here (keeps source clean)
```

### The CMakeLists.txt Recipe

Here is how we translate a Makefile logic to CMake:

```cmake
cmake_minimum_required(VERSION 3.10)
project(MyCalc)

# 1. Set Version
set(CMAKE_CXX_STANDARD 17)

# 2. Include Directories (Where are the .hpp files?)
# We don't use -I flags manually.
include_directories(include)

# 3. Create the Executable
# "Make an app called 'calculator' using these source files"
add_executable(calculator 
    src/main.cpp 
    src/simple_math.cpp
)
```

### 📝 Exercise: Compile the Project

If you have a terminal open in VS Code, navigate to the project folder inside this module and run:

```bash
cd 01_project 
mkdir build
cd build
cmake ..
make
./calculator
```

This is the "Out-of-source" build pattern. It keeps your .cpp files safe from temporary object files.